In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_3582/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104576,167601,https://www.washingtonpost.com/opinions/2023/0...,Opinion,\N,\N,2023-09-16 17:19:56.971854+00,2023-09-16 17:19:56.97186+00,NaN,\N,Custom,11.0,0.0,\N
104577,167602,https://docs.google.com/document/d/1X38dhFkdN0...,CLR research plan - Google Docs,\N,\N,2023-09-16 17:19:56.977151+00,2023-09-16 17:19:56.977157+00,NaN,\N,Custom,81.0,0.0,\N
104578,167603,https://progress.institute/a-serious-case-for-...,A Serious Case for Dynamic Scoring,\N,\N,2023-09-16 16:22:15.892742+00,2023-09-16 16:22:15.892742+00,NaN,\N,IFP,28.0,0.0,\N
104579,167604,https://www.slowboring.com/p/saturday-thread-049,Saturday Thread,\N,\N,2023-09-16 16:22:15.894958+00,2023-09-16 16:22:15.894958+00,NaN,\N,Yglesias,52.0,0.0,\N


In [2]:
links['added'].max()

'2023-09-16 17:19:56.977151+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
104573    167598
104574    167599
104575    167600
104576    167601
104577    167602
Name: id, Length: 13291, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/29
...11/29
...21/29
...29/29


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104605,167630,https://cltc.berkeley.edu/seeking-input-and-fe...,UC Berkeley AI Risk-Management Standards Profi...,\N,\N,2023-09-16 20:32:36.563107,2023-09-16 20:32:36.563113,\N,\N,Custom,40.0,0.0,\N
104606,167631,https://twitter.com/SSGamblers/status/17029254...,"Star Spangled Gamblers on X: ""New Episode: Sho...",\N,\N,2023-09-16 20:32:36.568698,2023-09-16 20:32:36.568703,\N,\N,Custom,49.0,0.0,\N
104607,167632,https://twitter.com/emollick/status/1703020667...,"Ethan Mollick on X: ""🚨We have a new working pa...",\N,\N,2023-09-16 20:32:36.574056,2023-09-16 20:32:36.574063,\N,\N,Custom,95.0,0.0,\N
104608,167633,https://www.washingtonpost.com/opinions/2023/0...,Opinion,\N,\N,2023-09-16 20:32:36.579428,2023-09-16 20:32:36.579434,\N,\N,Custom,81.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104605,167630,https://cltc.berkeley.edu/seeking-input-and-fe...,UC Berkeley AI Risk-Management Standards Profi...,\N,\N,2023-09-16 20:32:36.563107,2023-09-16 20:32:36.563113,\N,\N,Custom,40.0,0.0,\N
104606,167631,https://twitter.com/SSGamblers/status/17029254...,"Star Spangled Gamblers on X: ""New Episode: Sho...",\N,\N,2023-09-16 20:32:36.568698,2023-09-16 20:32:36.568703,\N,\N,Custom,49.0,0.0,\N
104607,167632,https://twitter.com/emollick/status/1703020667...,"Ethan Mollick on X: ""🚨We have a new working pa...",\N,\N,2023-09-16 20:32:36.574056,2023-09-16 20:32:36.574063,\N,\N,Custom,95.0,0.0,\N
104608,167633,https://www.washingtonpost.com/opinions/2023/0...,Opinion,\N,\N,2023-09-16 20:32:36.579428,2023-09-16 20:32:36.579434,\N,\N,Custom,81.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104605,167630,https://cltc.berkeley.edu/seeking-input-and-fe...,UC Berkeley AI Risk-Management Standards Profi...,\N,\N,2023-09-16 20:32:36.563107,2023-09-16 20:32:36.563113,\N,\N,Custom,40,0,\N
104606,167631,https://twitter.com/SSGamblers/status/17029254...,"Star Spangled Gamblers on X: ""New Episode: Sho...",\N,\N,2023-09-16 20:32:36.568698,2023-09-16 20:32:36.568703,\N,\N,Custom,49,0,\N
104607,167632,https://twitter.com/emollick/status/1703020667...,"Ethan Mollick on X: ""🚨We have a new working pa...",\N,\N,2023-09-16 20:32:36.574056,2023-09-16 20:32:36.574063,\N,\N,Custom,95,0,\N
104608,167633,https://www.washingtonpost.com/opinions/2023/0...,Opinion,\N,\N,2023-09-16 20:32:36.579428,2023-09-16 20:32:36.579434,\N,\N,Custom,81,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)